In [1]:
import pandas as pd
import json

def process_group_regional_data(file_path, output_json):
    cols = ['gname', 'region_txt', 'country_txt']
    df = pd.read_csv(file_path, encoding='ISO-8859-1', usecols=cols)

    group_mapping = {
        'ISIL': 'Islamic State of Iraq and the Levant (ISIL)',
        'taliban': 'Taliban',
        'SL': 'Shining Path (SL)'
    }

    output_data = {}

    for key, gtd_name in group_mapping.items():
        group_df = df[df['gname'] == gtd_name]
        if group_df.empty:
            output_data[key] = {"region": "Unknown", "data": []}
            continue

        top_region = group_df['region_txt'].mode()[0]
        region_df = group_df[group_df['region_txt'] == top_region]
        country_counts = region_df['country_txt'].value_counts().reset_index()
        country_counts.columns = ['country', 'count']

        replacements = {
            "Syria": "Syrian Arab Republic",
            "United States": "United States of America"
        }
        country_counts['country'] = country_counts['country'].replace(replacements)

        output_data[key] = {
            "region": top_region,
            "data": country_counts.to_dict(orient='records')
        }

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, indent=2)

process_group_regional_data('terrorism_dataset.csv', 'groups_regional_activity.json')